In [1]:
class GenomicException(Exception):
    pass


class ChromosomeNotFoundError(GenomicException):
    
    def __init__(self, chrom, genome, list_of_chroms):
        self.chrom = chrom
        self.genome = genome
        self.list_of_chroms = list_of_chroms
        super().__init__(f"Chromosome {self.chrom} is not "
                         f"in {self.genome} among these "
                         f"chromosomes: {', '.join(list_of_chroms)}")


class InconsistentChromosomesError(GenomicException):
    
    def __init__(self, one_chrom, other_chrom):
        self.one_chrom = one_chrom
        self.other_chrom = other_chrom
        super().__init__(f"Inconsistent chromosomes of "
                         f"comparing genomic ranges: "
                         f"{self.one_chrom} and {self.other_chrom}.")


class InconsistentGenomesError(GenomicException):

    def __init__(self, one_genome, other_genome):
        self.one_genome = one_genome
        self.other_genome = other_genome
        super().__init__(f"Inconsistent genomes of "
                         f"comparing genomic ranges: "
                         f"{self.one_genome} and {self.other_genome}.")

#class EmptyGenomicRangesListError(GenomicException):
#    
#    def __init__(self, range_list_instance):
#        self.range_list = range_list_instance
#s        super().__init__(f"Empty GenomicRangesList instance: {self.range_list}")


class GenomicCoordinatesError(GenomicException):
    
    def __init__(self, grange, chromosome):
        self.grange = grange
        self.chromosome = chromosome
        super().__init__(f"Coordinates of GenomicRange {self.grange} "
                         f"are out of chromosome with size {self.chromosome.size}")
        

In [137]:
def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr
    return start

In [138]:
def read_source(fileobj, total, target):
    read_chars = 0
    while read_chars < total:
        c = fileobj.read(1)
        if c == '\n':
            continue
        else:
            target.send(c)
            read_chars += 1
    print('Finished reading from file.')
    target.close()

@coroutine
def format_text_stream(target, linewidth=60):
    count = 0
    while True:
        c = (yield)
        count += 1
        target.send(c)
        if count % linewidth == 0:
            target.send('\n')

@coroutine
def write_text_stream(fileobj):
    c = '\n'
    try:
        while True:
            c = (yield)
            fileobj.write(c)
    except GeneratorExit:
        if c != '\n':
            fileobj.write('\n')
        print('Finished writing to file.')
        

In [139]:
test_line = "".join(['A' if i % 80 else '\n' for i in range(200)])
with open('test.txt', 'w') as outfile:
    outfile.write(test_line)
    outfile.write('\n')

In [140]:
!cat test.txt


AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


In [141]:
with open('test.txt', 'r') as infile:
    with open('test2.txt', 'w') as outfile:
        read_source(fileobj=infile, total=150,
                    target=format_text_stream(linewidth=60,
                                              target=write_text_stream(outfile)))

Finished reading from file.
Finished writing to file.


In [142]:
!cat -E test2.txt

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA$
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA$
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA$


In [143]:
def fasta_reformatter(infile, outfile, total, linewidth=60):
    read_counter = 0
    line_break = False
    while read_counter < total:
        c = infile.read(1)
        if c == "\n":
            continue
        else:
            read_counter += 1
            outfile.write(c)
            line_break = False
            if read_counter % linewidth == 0:
                outfile.write('\n')
                line_break = True
    if not line_break:
        outfile.write('\n')

In [144]:
with open('test.txt', 'r') as infile:
    with open('test3.txt', 'w') as outfile:
        fasta_reformatter(infile, outfile, 150, linewidth=11)

In [145]:
!cat -E test3.txt

AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAAAAAA$
AAAAAAA$


In [12]:
test_list = [[1, 2], [3, 4], [5, 6]]
sum(test_list, [])

[1, 2, 3, 4, 5, 6]

In [20]:
test_dict = {'key1': 'arg1', 'key2': 'arg2', 'key3': 'arg3'}
sum((['-' + key, value] for key, value in test_dict.items()), [])

['-key1', 'arg1', '-key2', 'arg2', '-key3', 'arg3']

In [147]:
from subprocess import Popen, PIPE

class GenomicRange:
    def __init__(self, chrom, start, end, strand,
                 genome=None, sequence_file_loc=None,
                 synteny=None, neighbours=None, **kwargs):
        self.chrom = chrom
        self.start = start
        self.end = end
        self.strand = strand
        self.genome = genome
        self._sequence_file_loc = sequence_file_loc
        self._sequence_header = None
        self.connections = {'synteny':, (GenomicRangesList([], self.genome)
                                         if synteny is None else synteny)
                            'neighbours': (GenomicRangesList([], self.genome)
                                           if neighbours is None else neighbours)}
    
    def __repr__(self):
        return f"GenomicRange({self.chrom}, {self.start}, {self.end}, {self.strand})"
    
    def __str__(self):
        return f"{self.chrom}\t{self.start}\t{self.end}\t{self.strand}"
    
    def distance(self, other):
        if self.chrom != other.chrom:
            raise InconsistentChromosomosesError(self.chrom, other.chrom)
        if self.genome != other.genome:
            raise InconsistentGenomesError(self.genome, other.genome)
        if self.end < other.start:
            return other.start - self.end
        if other.end < self.start:
            return other.end - self.start
        return 0
    
    def merge(self, other):
        if self.chrom != other.chrom:
            raise InconsistentChromosomosesError(self.chrom, other.chrom)
        if self.genome != other.genome:
            raise InconsistentGenomesError(self.genome, other.genome)
        start = min(self.start, other.start)
        end = max(self.end, other.end)
        return GenomicRange(chrom=self.chrom,
                            start=start,
                            end=end,
                            strand=".",
                            genome=self.genome)

    def align(self, connection='synteny', **kwargs):
        if connection not in self.connections:
            raise KeyError(f"There is no {connection} connections for the {self}.")
        for grange in self.connections[connection]:
            command = ['blastn',
                       '-task', 'blastn',
                       '-query', self._sequence_file_loc,
                       '-subject', grange._sequence_file_loc,
                       '-outfmt', '"7 std score"']
            add_args = sum((['-' + key, value]
                            for key, value in kwargs.items()),
                           [])
            with Popen(command + add_args, stdout=PIPE) as proc:
                alignment = proc.stdout.read()
                #TODO: add alignment module

    @property
    def sequence_header(self):
        if self._sequence_header is None:
            self._sequence_header = f"{self.chrom}:{self.start}" \
                                 f"-{self.end}{self.strand}"
        return self._sequence_header
    
    def merge_connections(self, distance=0, connection='synteny'):
        try:
            self.connections[connection] = self.connections[connection].merge(distance)
        except KeyError:
            raise KeyError(f"There is no {connection} connections for the {self}.")

In [46]:
from collections import namedtuple


ChromosomeLocation = namedtuple('ChromosomeLocation', 'size start')


class FastaSeqFile:
    def __init__(self, location):
        self.location = location
        self._chromsizes = dict()
        self._file_obj = None
    
    @property
    def chromsizes(self):
        if not self._chromsizes:
            with open(self.location, 'r') as infile:
                chrom, size, start = None, None, None
                line = infile.readline()
                while line:
                    if line.startswith(">"):
                        self._chromsizes[chrom] = ChromosomeLocation(size, start)
                        chrom = line.lstrip(">").rstrip().split()[0]
                        size = 0
                        start = None
                    else:
                        if start is None:
                            start = infile.tell() - len(line)
                        size += len(line.strip())
                    line = infile.readline()
                self._chromsizes[chrom] = ChromosomeLocation(size, start)
                del self._chromsizes[None]
        return self._chromsizes
    
    def __enter__(self):
        self._file_obj = open(self.location, 'r')
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self._file_obj.close()

    def get_fasta_by_coord(self, grange, outfile):
        try:
            _, start = self.chromsizes[grange.chrom].start
        except KeyError:
            raise ChromosomeNotFoundError(grange.chrom,
                                          self.location,
                                          self.chromsizes.keys())
        if grange.end > self.chromsizes[grange.chrom].size:
            raise GenomicCoordinatesError(grange, self.chromsizes[grange.chrom])
        self._file_obj.seek(start + grange.start, 0)
        outfile.write(">" + grange.sequence_header + "\n")
        fasta_reformatter(self._file_obj,
                          outfile,
                          total=(grange.end - grange.start),
                          linewidth=60)

In [42]:
!echo ">chrom1 test1\nAAA\nTTT\n\n>chrom2 test2\nGGGGG" > test.fasta

In [43]:
!cat test.fasta

>chrom1 test1
AAA
TTT

>chrom2 test2
GGGGG


In [47]:
fs = FastaSeqFile('test.fasta')

In [54]:
print(fs.chromsizes)

{'chrom1': ChromosomeLocation(size=6, start=14), 'chrom2': ChromosomeLocation(size=5, start=37)}


In [22]:
#fs.chromsizes = None

AttributeError: can't set attribute

In [3]:
from sortedcontainers import SortedKeyList

In [4]:
class GenomicRangesList(SortedKeyList):
    #TODO: add reading from file (bed, gff, gtf)
    #TODO: parsing orthodb-synteny output

    def __init__(self, collection=[], genome=None):
        super().__init__(iterable=collection, key=lambda x: (x.chrom, x.start, x.end))
        self.genome = genome
        self.source = FastaSeqFile(self.genome)
    
    def merge(self, distance=0):
        if len(self) == 0:
            raise EmptyGenomicRangesListError(self)
        new_range_list = GenomicRangesList([], self.genome)
        new_range_list.add(self[0])
        new_index = 0
        for grange in self[1:]:
            try:
                if abs(new_range_list[new_index].distance(grange)) <= distance:
                    new_range_list[new_index] = new_range_list[new_index].merge(grange)
                else:
                    new_range_list.add(grange)
                    new_index += 1
            except InconsistentChromosomesError:
                new_range_list.add(grange)
                new_index += 1
        return new_range_list
    
    def get_neighbours(self, other, distance=0):
        self_index, other_index = 0, 0
        current_self, current_other = self_index, other_index
        while self_index < len(self) and other_index < len(other):
            try:
                if self[current_self].distance(other[current_other]) < - distance:
                    current_other += 1
                elif abs(self[current_self].distance(other[current_other])) <= distance:
                    self[current_self].connections['neighbours'].add(other[current_other])
                    current_other += 1
                else:
                    self_index += 1
                    current_self = self_index
                    current_other = other_index
            except InconsistentChromosomesError:
                self_index = current_self
                other_index = current_other
                if self[self_index].chrom < other[other_index].chrom:
                    self_index += 1
                    current_self = self_index
                else:
                    other_index += 1
                    current_other = other_index
            except IndexError:
                if current_other >= len(other):
                    current_other = other_index
                    current_self += 1
                    self_index = current_self
                else:
                    self_index += 1
                    current_self = self_index
                    current_other = other_index
    
    def get_fasta(self, outfile, mode='split'):
        with self.source:
            if mode == 'split':
                for grange in self:
                    with open(outfile + grange.sequence_header + '.fasta', 'w') as output:
                        self.source.get_fasta_by_coord(grange, output)
            elif mode == 'bulk':
                with open(outfile + '.fasta', 'w') as output:
                    for grange in self:
                        self.source.get_fasta_by_coord(grange, output)
            else:
                raise ValueError("Invalid get fasta mode.")

In [5]:
sample_range_data = [[1, 10, 100, '+'],
                     [1, 50, 200, '-'],
                     [2, 200, 250, '+'],
                     [2, 10, 100, '-']]

In [6]:
sample_ranges = [GenomicRange(*i) for i in sample_range_data]

In [7]:
rc = GenomicRangesList(sample_ranges)

In [8]:
rc.add(GenomicRange(2, 25, 200, '+'))

In [11]:
print(rc)

RangeList([GenomicRange(1, 10, 100, +), GenomicRange(1, 50, 200, -), GenomicRange(2, 10, 100, -), GenomicRange(2, 25, 200, +), GenomicRange(2, 200, 250, +)], key=<function RangeList.__init__.<locals>.<lambda> at 0x7f81a02bfae8>)


In [9]:
a = 5
b = (a
     if a == 5 else 10)